### PRE REQUISITES

SINGLE PARITY DETECTION FUCTIONS

In [1]:
def Even_parity_send(orig_data):    
    o = orig_data.count("1")
    if(o%2 != 0):
        return orig_data + "1"
    else:
        return orig_data + "0"
    
def Even_parity_receive(received_data):
    o = received_data.count("1")
    if(o%2 == 0):
        return True
    return False
    
def Odd_parity_send(orig_data):    
    o = orig_data.count("1")
    if(o%2 == 0): 
        return orig_data + "1"
    else:
        return orig_data + "0"
    
def Odd_parity_receive(received_data):
    o = received_data.count("1")
    if(o%2 != 0):
        return True
    return False

REQUIRED LIBRARIES

In [2]:
import time
import random

### STOP AND WAIT

In [4]:
data = input("Enter the data : ")

dt = list(data)
while(len(dt)%5 != 0):      # Fixed frame size ==> 5
    dt.insert(0,"0")
data = ''.join(dt)

send_frames = []       # contains encoded data to be sent by send
receive_frames = []    # contains decoded data to be received by receiver

print("\nFrames ==> ", end = "")
for i in range(0,len(data),5):        
    send_frames.append(data[i:i+5])
    print(data[i:i+5], end = " ")      # print the data splitted into frames
print("\n\n")

for i in range(len(send_frames)):      # encode the data in send_frames 
    send_frames[i] = Even_parity_send(send_frames[i])

flag = False
frameno = len(send_frames)
Sn = 1
ack = 1
while True:
    if(frameno == 0):     # if all frames sent, then stop the program
        break
    beg_time = time.perf_counter()
    print("Sending Frame", Sn, " ==> ", send_frames[Sn-1])    # indicates the data is sent
    while True:
        if random.randrange(0,4)%5 == 0 :            # Used to create delay in time during the sending process
            break
        else:
            if((time.perf_counter() - beg_time) > 4):    # Don't wait more than 4 seconds, resend it
                print("Re",end = "")
                flag = True
                break
            time.sleep(2)      # time taken for acknowledgement pack to reach sender
            
    if(not Even_parity_receive(send_frames[Sn-1])):     # detects if any error
        flag = True

    if(flag):            # if flag, then retransmission of data
        flag = False
        continue
        
    Sn += 1             # else prepare for acknowledgement and send next frame 
    ack += 1
    
    print("Receiving Ack",ack,"\n")
    receive_frames.append(send_frames[Sn-2][:-1])     # decode received data
    time.sleep(0.5)
    frameno -= 1

print("\nReceived data ==> ", end = "")     # print the data
for i in receive_frames:
    print(str(i), end = " ")
    
#0111011101001111010100010101
#try the above as input

Enter the data : 0111011101001111010100010101

Frames ==> 00011 10111 01001 11101 01000 10101 


Sending Frame 1  ==>  000110
ReSending Frame 1  ==>  000110
ReSending Frame 1  ==>  000110
Receiving Ack 2 

Sending Frame 2  ==>  101110
ReSending Frame 2  ==>  101110
Receiving Ack 3 

Sending Frame 3  ==>  010010
ReSending Frame 3  ==>  010010
Receiving Ack 4 

Sending Frame 4  ==>  111010
ReSending Frame 4  ==>  111010
Receiving Ack 5 

Sending Frame 5  ==>  010001
Receiving Ack 6 

Sending Frame 6  ==>  101011
ReSending Frame 6  ==>  101011
ReSending Frame 6  ==>  101011
Receiving Ack 7 


Received data ==> 00011 10111 01001 11101 01000 10101 

### GO BACK N ARQ

In [49]:
data = input("Enter the data : ")

dt = list(data)
while(len(dt)%5 != 0):      # Fixed frame size ==> 5
    dt.insert(0,"0")
data = ''.join(dt)

send_frames = []       # contains encoded data to be sent by send
receive_frames = []    # contains decoded data to be received by receiver

print("\nFrames ==> ", end = "")
for i in range(0,len(data),5):        
    send_frames.append(data[i:i+5])
    print(data[i:i+5], end = " ")      # print the data splitted into frames
print("\n")

for i in range(len(send_frames)):      # encode the data in send_frames 
    send_frames[i] = Even_parity_send(send_frames[i])
    
Sn = 1
Sf = 1
Sw = 3
ack = 2
flag = False
no_of_frames = len(send_frames)

while(True):
    
    if Sf == no_of_frames+1 :     # if all frames sent, then stop the program
        break
        
    n = random.randrange(Sw+1)     # no of frames to be transmitted
    while Sf+n > no_of_frames :
        n = random.randrange(Sw+1)
        
    print("\ncurrent window",end=" ==> ")
    for i in range(Sf,Sf+Sw):
        print(((i-1)%(no_of_frames))+1,end=" ")
    print("\n")
    
    beg_time = time.perf_counter()
    for i in range(Sf,Sf+n+1):
        print("Sending frame",Sn,"==>",send_frames[Sn-1])
        Sn += 1

    for i in range(Sf,Sf+n+1):
        beg_time = time.perf_counter()
        while True:
            if random.randrange(0,4)%5 == 0 :
                if Even_parity_receive(send_frames[i-1]):
                    print("Receiveing Ack",ack)
                    receive_frames.append(send_frames[i-1][:-1])
                    ack += 1
                    Sf += 1
                    break
            else:
                if((time.perf_counter() - beg_time) > 3):    # Don't wait more than 3 seconds, resend it
                    print("\nRetransmitting ",end="")
                    flag = True
                    ack = Sf+1
                    Sn = Sf
                    break
                time.sleep(1)      # time taken for acknowledgement pack to reach sender
                
        if flag:
            flag = False
            break
            
print("\nReceived data ==> ", end = "")     # print the data
for i in receive_frames:
    print(str(i), end = " ")
       
#0111011101001111010100010101
#try the above as input

Enter the data : 0111011101001111010100010101

Frames ==> 00011 10111 01001 11101 01000 10101 


current window ==> 1 2 3 

Sending frame 1 ==> 000110
Sending frame 2 ==> 101110

Retransmitting 
current window ==> 1 2 3 

Sending frame 1 ==> 000110
Sending frame 2 ==> 101110
Sending frame 3 ==> 010010
Receiveing Ack 2
Receiveing Ack 3

Retransmitting 
current window ==> 3 4 5 

Sending frame 3 ==> 010010
Sending frame 4 ==> 111010
Receiveing Ack 4

Retransmitting 
current window ==> 4 5 6 

Sending frame 4 ==> 111010
Receiveing Ack 5

current window ==> 5 6 1 

Sending frame 5 ==> 010001
Sending frame 6 ==> 101011

Retransmitting 
current window ==> 5 6 1 

Sending frame 5 ==> 010001
Receiveing Ack 6

current window ==> 6 1 2 

Sending frame 6 ==> 101011
Receiveing Ack 7

Received data ==> 00011 10111 01001 11101 01000 10101 

### SELECTIVE REPEAT ARQ

In [5]:
data = input("Enter the data : ")

dt = list(data)
while(len(dt)%5 != 0):      # Fixed frame size ==> 5
    dt.insert(0,"0")
data = ''.join(dt)

send_frames = []       # contains encoded data to be sent by send
receive_frames = []    # contains decoded data to be received by receiver

print("\nFrames ==> ", end = "")
for i in range(0,len(data),5):        
    send_frames.append(data[i:i+5])
    print(data[i:i+5], end = " ")      # print the data splitted into frames
print("\n")

for i in range(len(send_frames)):      # encode the data in send_frames 
    send_frames[i] = Even_parity_send(send_frames[i])
    
Sn = 1
Sf = 1
Sw = 3
Rn = 2
flag = False
no_of_frames = len(send_frames)

while(True):
    
    if Sf == no_of_frames+1 :     # if all frames sent, then stop the program
        break
        
    n = random.randrange(Sw)     # no of frames to be transmitted
    while Sf+n > no_of_frames :
        n = random.randrange(Sw)
        
    print("\ncurrent window",end=" ==> ")
    for i in range(Sf,Sf+Sw):
        print(((i-1)%(no_of_frames))+1,end=" ")
    print("\n")
    
    beg_time = time.perf_counter()
    for i in range(Sf,Sf+n+1):                      # frames are being sent
        print("Sending frame",Sn,"==>",send_frames[Sn-1])
        Sn += 1

    for i in range(Sf,Sf+n+1):
        beg_time = time.perf_counter()
        while True:
            if random.randrange(0,4)%5 == 0 :       # if zero is generated, acknowledement sent
                print("Receiveing Ack",Rn)
                receive_frames.append(send_frames[i-1][:-1])
                Rn += 1
                Sf += 1
                break
            elif random.randrange(0,4)%5 == 1 :     # if one is generated, -ve ack is sent
                print("    Receiving NAK", Rn-1)
                print("    Resending Frame",Rn-1)
                time.sleep(1.5)
                print("    Receiving ACK",Rn)
                receive_frames.append(send_frames[i-1][:-1])
                Sf += 1
                Rn += 1
                break
                
            else:
                if((time.perf_counter() - beg_time) > 3):    # Don't wait more than 3 seconds, resend it
                    print("\nTimeout")
                    print("Retransmitting",end = " ")
                    flag = True
                    Sn = Sf
                    break
                time.sleep(1)      # time taken for acknowledgement pack to reach sender
                
        if flag:
            flag = False
            break
            
print("\nReceived data ==> ", end = "")     # print the data
for i in receive_frames:
    print(str(i), end = " ")

#0111011101001111010100010101
#try the above as input

Enter the data : 0111011101001111010100010101

Frames ==> 00011 10111 01001 11101 01000 10101 


current window ==> 1 2 3 

Sending frame 1 ==> 000110
Sending frame 2 ==> 101110
Sending frame 3 ==> 010010
    Receiving NAK 1
    Resending Frame 1
    Receiving ACK 2
Receiveing Ack 3
Receiveing Ack 4

current window ==> 4 5 6 

Sending frame 4 ==> 111010
Sending frame 5 ==> 010001
Sending frame 6 ==> 101011
    Receiving NAK 4
    Resending Frame 4
    Receiving ACK 5
Receiveing Ack 6
    Receiving NAK 6
    Resending Frame 6
    Receiving ACK 7

Received data ==> 00011 10111 01001 11101 01000 10101 